In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os, time, re, random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
!pip3 install -q tensorflow_gpu>=2.0

import tensorflow as tf
print(tf.__version__)

ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.
2.1.0


In [4]:
!pip3 install -q ktrain

     |████████████████████████████████| 102kB 2.9MB/s 
     |████████████████████████████████| 6.7MB 10.9MB/s 
     |████████████████████████████████| 1.0MB 43.4MB/s 
     |████████████████████████████████| 245kB 57.0MB/s 
     |████████████████████████████████| 1.8MB 42.4MB/s 
     |████████████████████████████████| 481kB 70.2MB/s 
     |████████████████████████████████| 3.1MB 40.3MB/s 
     |████████████████████████████████| 870kB 53.3MB/s 
     |████████████████████████████████| 1.0MB 24.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [5]:
import ktrain
from ktrain import text

using Keras version: 2.2.4-tf


In [0]:
d = "/content/drive/My Drive/Colab Notebooks/Datasets/Personal Case Study/Search Engine/"
model_path = d+"Models/"

In [7]:
def load_data():
    """
    this will load the data
    """
    df = pd.read_csv(d+"data.tsv", sep = "\t", header = None, low_memory = True)
    col_names = ["que_no", "question", "answer", "label", "ans_no"]
    print("Before number column in dataset are : ", df.shape[1])
    df.columns = col_names
    # Note: ans_no, que_no column is not useful for this probelm, so I'm droping this column
    df.drop(["que_no","ans_no"], axis = 1, inplace = True)
    print("After droping number columns in dataset are: ", df.shape[1])
    print("Number of datapoints in dataset : {:,}".format(df.shape[0]))
    return df

try:
    stop_words = set(stopwords.words("english"))
except:
    nltk.download('stopwords')
    stop_words = set(stopwords.words("english"))
#removing "not" stop word from stop_words
stop_words = stop_words - {"not", "what", "when", "where", "how", "which"}

##########################################################################################################
#################################### function to pre-process the text ####################################
##########################################################################################################
def text_process(row):
    """
    this function is for pre-processing the text.
    exppand the sentences, remove stopwords, removes html, remove extra space etc.
    """
    try:
        text = row
        text = str(text).lower()
        porter = PorterStemmer()

        #expansion
        text = text.replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
        .replace("n't", " not").replace("what's", "what is").replace("it's", "itis")\
        .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
        .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
        .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar")\
        .replace("€", " euro ").replace("'ll", " will")

        text = re.sub(r"<.*?>","", text) # removes the htmltags: https://stackoverflow.com/a/12982689

        #special character removal
        text = re.sub('[^a-zA-Z0-9\n]', ' ', text)
        #extra space removal
        text = re.sub('\s+',' ', text)

        # stopword removal
        text_to_words = []
        for word in text.split():
            if word not in stop_words:
                text_to_words.append(word)
            else:
                continue
        text = " ".join(text_to_words)

        # stemm the words of sentence
        text = porter.stem(text)

        return text

    except:
        print("There is no value in comment_text, so returnin 'nan'")
        
        return np.nan

##########################################################################################################
#################################### function to design the features  ####################################
##########################################################################################################

def hand_craft_features(data_df):
    """
    this creates the hand crafted features.
    """
    # lenth features
    data_df["que_len"] = data_df["question"].apply(lambda x:len(x))
    data_df["ans_len"] = data_df["answer"].apply(lambda x:len(x))
    data_df["que_word"] = data_df["question"].apply(lambda x:len(x.split()))
    data_df["ans_word"] = data_df["answer"].apply(lambda x:len(x.split()))
    data_df["unique_word_que"] = data_df["question"].apply(lambda x:len(set(x.split())))
    data_df["unique_word_ans"] = data_df["answer"].apply(lambda x:len(set(x.split())))
    #ratio features
    data_df["ratio_q_a_len"] = data_df["que_len"]/data_df["ans_len"]
    data_df["ratio_q_a_word"] = data_df["que_word"]/data_df["ans_word"]
    data_df["ratio_q_a_unique_word"] = data_df["unique_word_que"]/data_df["unique_word_ans"]
    data_df["ration_qword_alen"] = data_df["que_word"]/data_df["ans_len"]
    data_df["ratio_aword_qlen"] = data_df["ans_word"]/data_df["que_len"]
    data_df["ratio_unique_qword_aword"] = data_df["unique_word_que"]/data_df["ans_word"]
    data_df["ratio_unique_aword_qword"] = data_df["unique_word_ans"]/data_df["que_word"]
 
    df.replace([np.inf], np.nan, inplace = True)
    data_df.fillna(0, inplace=True)
    
    return data_df

##########################################################################################################
#################################### function to plot confudion matrix   #################################
##########################################################################################################
from sklearn.metrics import confusion_matrix
def plot_C_P_R_matrix(y_true, y_pred, name):
    """
    Funtion to plot confusion, precision and recall matrix
    """
    C = confusion_matrix(y_true, y_pred) #for confusion matrix
    # for presion matrix: take the sum column wise, i.e. column normalization
    P = C/C.sum(axis = 0) # it'll have column sum = 1
    #for recall matrix: take the sum row wise, i.e. row normalization
    R = (C.T/C.sum(axis = 1)).T #it'll have row sum = 1

    fig, (ax1, ax2, ax3)  = plt.subplots(1,3, figsize= (19,4))

    #https://seaborn.pydata.org/generated/seaborn.heatmap.html
    sns.heatmap(C, annot = True, fmt = 'd', annot_kws={"size":15}, ax = ax1)
    ax1.set_title("Confusion Matrix", fontsize = 15)
    ax1.set_xlabel("Predicted", fontsize = 12)
    ax1.set_ylabel("Actual", fontsize = 12)

    sns.heatmap(P, annot = True, fmt = '.3f', annot_kws={"size":15}, ax = ax2)
    ax2.set_title("Precision Matrix", fontsize = 15)
    ax2.set_xlabel("Predicted", fontsize = 12)
    ax2.set_ylabel("Actual", fontsize = 12)

    sns.heatmap(R, annot = True, fmt = '.3f', annot_kws={"size":15}, ax = ax3)
    ax3.set_title("Recall Matrix", fontsize = 15)
    ax3.set_xlabel("Predicted", fontsize = 12)
    ax3.set_ylabel("Actual", fontsize = 12)

    plt.show()

########################################################################################
#######################    function to plot Confusion matrix     #######################
########################################################################################
def plot_confusion_matrix(y_train_pred, y_cv_pred, y_test_pred, y_train, y_cv, y_test, thres):
    """
    funtion to plot the precison, recall and confusion matrix for train, cv and test data
    """
    tr_pred = np.where(y_train_pred >= thres, 1, 0)
    cv_pred = np.where(y_cv_pred >= thres, 1, 0)
    te_pred = np.where(y_test_pred >= thres, 1, 0)   

    print(" "*30, "*"*20, "For Train data", "*"*20)
    plot_C_P_R_matrix(y_train.values.flatten(), tr_pred, name = "Train")

    print(" "*30, "*"*20, "For CV data", "*"*20)
    plot_C_P_R_matrix(y_cv.values.flatten(), cv_pred, name = "CV")

    print(" "*30, "*"*20, "For Test data", "*"*20)
    plot_C_P_R_matrix(y_test.values.flatten(), te_pred, name = "Test")

########################################################################################################
############################      function to calculate the MRR        #################################
########################################################################################################
from sklearn.metrics import label_ranking_average_precision_score
def get_mrr(y_true, y_pred):
    """
    to calcuate the mrr.
    it take the y_true and y_pred and make list of lists such that
    for each question ten answer result in list.
    """
    y_true = np.array([y_true[i*10:(i+1)*10] for i in range(len(y_true)//10)])
    y_pred = np.array([y_pred[i*10:(i+1)*10] for i in range(len(y_pred)//10)])
    return label_ranking_average_precision_score(y_true, y_pred)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def train(train_it = False):
    if train_it:
        if not os.path.isfile(d+"Processed Data/text_processed.csv"):
            print("processed data is not in drive, let's create...")
            #laod the data
            df = load_data()
            # pre-process text data
            df.loc[:,"question"] = df["question"].apply(text_process)
            df.loc[:,"answer"] = df["answer"].apply(text_process)
            # relacing such question and answers with np.Nan and then droping them
            df.replace(["", "null"], np.nan, inplace = True)
            df.dropna(axis = 0, how = "any", inplace= True)
            # drop duplicates
            df.drop_duplicates(subset = ["question", "answer"], keep = False, inplace = True)

            # Let's remove the questions that have more than 1 correct answer
            # https://stackoverflow.com/a/43501282/12005970
            a = df.groupby('question')\
                    .agg({'answer': 'count', 'label': 'sum'})\
                    .reset_index()\
                    .rename(columns={'answer':'total answers', 'label': 'correct answers'})
            # questions that have more than 1 correct answer
            multiple_correct_answers_df = a[(a['correct answers']>1) 
                                            or (a['correct answers'] == 0)
                                            or (a['total answers'] < 10)
                                            or (a['total answers'] > 10)]
            df.drop(df[df['question'].isin(multiple_correct_answers)].index, axis = 0, inplace=True)
            df.reset_index(inplace=True, drop= True)

            # design text based features
            df = hand_craft_features(df)
            df = df.head(500000)
        else:
            print("laoding the processed dataset from drive...")
            df = pd.read_csv(d+"Processed Data/text_processed.csv", low_memory=True)
            df = df.head(500000)
        
        # split the dataset
        print("splitting the data...")
        df_train = df.iloc[0:300000, :] 
        df_cv    = df.iloc[300000:400000, :]
        df_test  = df.iloc[400000:, :]
        
        print("Converting text into vectors...")
        # definig the model name
        MODEL_NAME = 'distilbert-base-uncased'
        # getting the Transformer
        t = text.Transformer(MODEL_NAME, maxlen = 200, classes=[0,1])
        # pre-prcoessing the data for training the model 
        trn = t.preprocess_train(list(df_train['question'] + ['[SEP]'] + df_train['answer']),
                                df_train['label'].values)
        val = t.preprocess_test(list(df_cv['question'] + ['[SEP]'] + df_cv['answer']),
                                df_cv['label'].values)
        model = t.get_classifier()
        # creating the batchs of data created in above lines
        learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=64)
        # model trainig
        learner.fit_onecycle(1*10**(-4), 2)
        #save the predictor
        predictor = ktrain.get_predictor(learner.model, preproc=t)
        predictor.save(model_path+'bert/')
    else:
        print('model is already trained')

In [9]:
train()

model is already trained


## **`final1()` fnction**

* This function will take the data in (X,Y) format give the performance of model.
* It is like evaluating function for the model.

**Note:** Before sending (X,Y) in function make sure that they follow these conditions-
* Each question must have only 10 answer and only one of them should be correct.
* First 10 row will correspond first question, next 10 rows will correspond to next question, next 10 rows will correspond to third question and so on. 

In [0]:
#load the trained model
predictor = ktrain.load_predictor(model_path+'bert/')

In [11]:
# loading the dataset. it satisfies above two condition
df = pd.read_csv(d+"Processed Data/text_processed.csv", low_memory=True)
df = df.head(500000)
# split the dataset
print("splitting the data...")
df_train = df.iloc[0:300000, :] 
df_cv    = df.iloc[300000:400000, :]
df_test  = df.iloc[400000:, :]

splitting the data...


In [0]:
X = df_cv[['question', 'answer']]
Y = df_cv['label']

In [0]:
def final1(X,Y, already_pred = True):
    '''
    this function will take (X,Y) and returns the MRR.
    this alos print the confusion, precision and recall matrix.
    X: dataframe: it has two columns [['question','answer']]
    Y: it can dataframe, or panadas series. it has the class labels
    already_pred: if predicted class-proabilities is stored in disk or not
    X and Y must satisfy this two condition, other actual result be differe
    1. each question must have only 10 answers and only of them should be correct.
    2. First 10 row will correspond first question, next 10 rows will correspond to next question,
     next 10 rows will correspond to third question and so on.
    '''
    if already_pred:
        print('for given X model prediction is saved. So loading from the disk')
        df = pd.read_csv(d+'cv_pred.csv')
        mrr = get_mrr(df['actual_label'].values, df['class1_proba'].values)
    else:
        # prediction
        Y_pred = predictor.predict_proba(list(X['question'] + ['[SEP]'] + X['answer']))[:,1]
        # calculating the MRR
        mrr = get_mrr(Y.values.flatten(), Y_pred.flatten())
    return mrr

In [14]:
mrr = final1(X,Y)
print('MRR = {:.4f}'.format(mrr))

for given X model prediction is saved. So loading from the disk
MRR = 0.5607


## **`final2()` function**

* This function will take the `X` only predict whether the question is correct or not.

In [0]:
def final2(X):
    '''
    this function take `X` and return predicted Y {0,1}
    X: is dataframe: [['question','answer']]
    function will predict the label of question and answer pair.

    this function will return the class and class-probobabilities of each class
    '''
    #connvert text into sequence
    print('text to sequnce...')
    Y_pred = predictor.predict_proba(list(X['question'] + ['[SEP]'] + X['answer']))
    label = np.where(Y_pred[:,1]>=0.2,1,0)
    return label, Y_pred

In [45]:
# as one datapoints
X = df_cv[['question', 'answer', 'label']].head(1)
label, Y_pred = final2(X)
print(f'for Question = {X.question.values[0]}\n\
    Answer = {X.answer.values[0]}\n\
    Actual class label = {X.label.values[0]}\n\
    Predicted class label = {label[0]}')

text to sequnce...


for Question = cast unbreakable kimmy schmidt
    Answer = unbreakable kimmy schmidt cast breaks season 2 finale shockers stars ellie kemper tituss burgess jane krakowski carol kane discuss season biggest moments preview what ahead season three everyone unbreakable kimmy schmidt ends season two different place where started
    Actual class label = 0
    Predicted class label = 0


In [47]:
# for multiple inputs
X = df_cv[['question', 'answer', 'label']].head(10)
label, Y_pred = final2(X)
for i in range(10):
    print(f'for Question = {X.question.values[i]}\n\
    Answer = {X.answer.values[i]}\n\
    Actual class label = {X.label.values[i]}\n\
    Predicted class label = {label[i]}')

text to sequnce...


for Question = cast unbreakable kimmy schmidt
    Answer = unbreakable kimmy schmidt cast breaks season 2 finale shockers stars ellie kemper tituss burgess jane krakowski carol kane discuss season biggest moments preview what ahead season three everyone unbreakable kimmy schmidt ends season two different place where started
    Actual class label = 0
    Predicted class label = 0
for Question = cast unbreakable kimmy schmidt
    Answer = unbreakable kimmy schmidt unbreakable kimmy schmidt american web television sitcom created tina fey robert carlock starring ellie kemper title role streamed netflix since march 6 2015 originally set 13 episode first season nbc spring 2015 show sold netflix given two season order
    Actual class label = 0
    Predicted class label = 1
for Question = cast unbreakable kimmy schmidt
    Answer = hollywood memorable movies actually remakes films across globe check international films hits inspired
    Actual class label = 0
    Predicted class label = 0
fo

In [49]:
from sklearn.metrics import classification_report
# for training data
# loading saved the predicted values
df = pd.read_csv(d+'train_pred.csv') 
print(classification_report(df['actual_label'], np.where(df['class1_proba'].values>=0.2,1,0)))

              precision    recall  f1-score   support

           0       0.97      0.85      0.91    270000
           1       0.36      0.75      0.49     30000

    accuracy                           0.84    300000
   macro avg       0.67      0.80      0.70    300000
weighted avg       0.91      0.84      0.87    300000



In [50]:
# for CV data
# loading saved the predicted values
df = pd.read_csv(d+'cv_pred.csv') 
print(classification_report(df['actual_label'], np.where(df['class1_proba'].values>=0.2,1,0)))

              precision    recall  f1-score   support

           0       0.94      0.80      0.87     90000
           1       0.24      0.55      0.33     10000

    accuracy                           0.78    100000
   macro avg       0.59      0.68      0.60    100000
weighted avg       0.87      0.78      0.81    100000



In [51]:
# for test data
# loading saved the predicted values
df = pd.read_csv(d+'test_pred.csv') 
print(classification_report(df['actual_label'], np.where(df['class1_proba'].values>=0.2,1,0)))

              precision    recall  f1-score   support

           0       0.94      0.84      0.89     90000
           1       0.27      0.53      0.36     10000

    accuracy                           0.81    100000
   macro avg       0.61      0.69      0.63    100000
weighted avg       0.87      0.81      0.84    100000

